# Preliminary Analysis

### Data Cleaning Code
Code for cleaning and processing your data. Include a data dictionary for your transformed dataset.

- Data Dictionary for Air Quality
    - **indicator id:** id for each name
    - **name:** classify the sample in the air
    - **measure:** how the indicator is measured
    - **measure info:** information about the measure
    - **geo type name:** geography type, UHF stands for United Hospital Fund neighborhoods
    - **geo place name:** neighborhood name
    - **time period:** time frame
    - **start_date:** date started
    <br><br>
- Data Dictionary for Traffic Volume
    - **requestId:** unique id generated for each counts request
    - **boro:** lists which of the five diviions of New York City the location is within
    - **vol:** total sum of count collected within 15 minute increments
    - **segmentId:** The ID that idenifies each segment of a street
    - **wktgeom:** Geometry point of the location
    - **street:** street name of where traffic happened
    - **fromst:** start street of traffic
    - **tost:** end street where traffic volume was located
    - **direction:** text-based direction of traffic where the count took place
    - **date_time:** date at which it took place
    <br><br>
- Data Dictionary for 2020 mobility Dataset
    - **sub_region_2** which county it is
    - **date** date during recording
    - **retail_and_recreation_percent_change_from_baseline** mobility trends for places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.
    - **grocery_and_pharmacy_percent_change_from_baseline** mobility trends for places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies
    - **parks_percent_change_from_baseline** mobility trends for places like national parks, public beaches, marinas, dog parks, plazas, and public gardens
    - **transit_stations_percent_change_from_baseline** mobility trends for places like public transport hubs such as subway, bus, and train stations
    - **workplaces_percent_change_from_baseline** mobility trend for places of work
    - **residential_percent_change_from_baseline** mobility trends for places of residence
    
### Exploratory Analysis
Describe what work you have done so far and include the code. This may include descriptive statistics, graphs and charts, and preliminary models.

- We removed some columns that were irrelevant to what we want to predict as well as combine some columns that would fit together, such as the date and time.


### Challenges
Describe any challenges you've encountered so far. Let me know if there's anything you need help with!

- There were some challenges in figuring out what sort of data was necessary to include for our problem as it was targeted in New York City. 
- Figuring out the transformations to use on each dataset was also a challenge since there were many columns for each dataset and we had to find the ones that weren't relevant to our problem.
- There are some issues for the columns right now where there are some, such as segmentId in the Traffic Volume dataset where we are currently unsure if it's useful to keep or remove.
- Dealing with large datasets efficiently

### Future Work
Describe what work you are planning to complete for the final analysis.

- Future work includes using the cleaned data to use as inputs for models suited for classification such as Logisitc Regression and Linear Regression. 
- Make predictions using the models trained to obtain the accuracy scores to answer our questions
- Find the best model for accuracy as well as graph/chart the data to further understand it for future predictions.

### Contributions
Describe the contributions that each group member made.
- **Daniel Aguilar-Rodriguez**
    - Researched and acquired datasets
    - Helped present ideas during brainstorming session
    - Created jupyter notebook and helped clean datasets
    - Helped transform datasets and removed columns irrelevant to our work
    <br><br>
- **Jia Cong Lin**
    - Helped present ideas during brainstorming session
    - Helped define necessary columns for the mobility dataset
    - Assisted in determining columns to clean and define 
    <br><br>
- **Anvinh Truong**
    - Helped clean and define some columns for the datasets and dictionary
    - Helped present ideas during brainstorming session
    - Assisted in thinking of procedure to clean data columns

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import datetime

In [2]:
def download_data(csv_name):
    url_dict = {'air_quality': 'https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.csv', 
                'mobility_global': 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', 
                'traffic_volume': 'https://data.cityofnewyork.us/api/views/7ym2-wayt/rows.csv'}
    
    response = requests.get(url_dict[csv_name])
    path = f'datasets/{csv_name}.csv'
    with open(path, 'wb') as f:
        f.write(response.content)

In [3]:
def csv_exists(csv_name):
    path = f'datasets/{csv_name}.csv'
    file_exists = os.path.exists(path)
    return file_exists

In [4]:
def create_df(csv_name):
    if not csv_exists(csv_name):
        download_data(csv_name)
    path = f'datasets/{csv_name}.csv'
    df = pd.read_csv(path)
    return df

In [5]:
def mkdir_if_not_exist():
    directory = 'datasets'
    if not os.path.exists(f'{directory}/'):
        os.mkdir(directory)

In [6]:
def create_all_df(csv_names):
    mkdir_if_not_exist()
    df_list = []
    
    for csv_name in csv_names:
        print(f'Creating {csv_name} df')
        df = create_df(csv_name)
        df_list.append(df)
        
    return df_list

In [7]:
csv_names = ['air_quality', 'traffic_volume']

air_quality, traffic_volume = create_all_df(csv_names)

Creating air_quality df
Creating traffic_volume df


## Air Quality Dataset Cleaning

In [8]:
print(air_quality.isnull().sum() / len(air_quality))

Unique ID         0.0
Indicator ID      0.0
Name              0.0
Measure           0.0
Measure Info      0.0
Geo Type Name     0.0
Geo Join ID       0.0
Geo Place Name    0.0
Time Period       0.0
Start_Date        0.0
Data Value        0.0
Message           1.0
dtype: float64


In [9]:
air_quality = air_quality.drop(['Message'], axis=1)
print(air_quality.isnull().sum() / len(air_quality))

Unique ID         0.0
Indicator ID      0.0
Name              0.0
Measure           0.0
Measure Info      0.0
Geo Type Name     0.0
Geo Join ID       0.0
Geo Place Name    0.0
Time Period       0.0
Start_Date        0.0
Data Value        0.0
dtype: float64


In [10]:
print(air_quality.nunique() / len(air_quality))

Unique ID         1.000000
Indicator ID      0.001365
Name              0.001179
Measure           0.000496
Measure Info      0.000496
Geo Type Name     0.000310
Geo Join ID       0.004466
Geo Place Name    0.007071
Time Period       0.002791
Start_Date        0.002233
Data Value        0.253443
dtype: float64


In [11]:
air_quality = air_quality.drop(['Unique ID'], axis=1)
print(air_quality.shape)
print(air_quality.nunique() / len(air_quality))

(16122, 10)
Indicator ID      0.001365
Name              0.001179
Measure           0.000496
Measure Info      0.000496
Geo Type Name     0.000310
Geo Join ID       0.004466
Geo Place Name    0.007071
Time Period       0.002791
Start_Date        0.002233
Data Value        0.253443
dtype: float64


In [12]:
air_quality = air_quality.drop(['Geo Join ID'], axis=1)
print(air_quality.shape)
print(air_quality.nunique() / len(air_quality))

(16122, 9)
Indicator ID      0.001365
Name              0.001179
Measure           0.000496
Measure Info      0.000496
Geo Type Name     0.000310
Geo Place Name    0.007071
Time Period       0.002791
Start_Date        0.002233
Data Value        0.253443
dtype: float64


In [13]:
air_quality.dtypes

Indicator ID        int64
Name               object
Measure            object
Measure Info       object
Geo Type Name      object
Geo Place Name     object
Time Period        object
Start_Date         object
Data Value        float64
dtype: object

In [14]:
air_quality.nunique()

Indicator ID        22
Name                19
Measure              8
Measure Info         8
Geo Type Name        5
Geo Place Name     114
Time Period         45
Start_Date          36
Data Value        4086
dtype: int64

In [15]:
air_quality['Time Period'].unique()

array(['Summer 2013', 'Summer 2014', 'Winter 2008-09', 'Summer 2009',
       'Summer 2010', 'Summer 2011', 'Summer 2012', 'Winter 2009-10',
       '2005-2007', '2013', '2005', '2009-2011', 'Winter 2010-11',
       'Winter 2011-12', 'Winter 2012-13', 'Annual Average 2009',
       'Annual Average 2010', 'Annual Average 2011',
       'Annual Average 2012', 'Annual Average 2013', '2015',
       'Winter 2013-14', 'Annual Average 2014', '2011', 'Winter 2014-15',
       '2016', 'Annual Average 2015', 'Summer 2015', 'Winter 2015-16',
       'Summer 2016', 'Annual Average 2016', 'Summer 2017', '2012-2014',
       'Summer 2018', 'Annual Average 2017', 'Summer 2019',
       'Winter 2016-17', 'Annual Average 2018', 'Winter 2017-18',
       '2015-2017', 'Summer 2020', 'Annual Average 2019',
       'Winter 2018-19', 'Annual Average 2020', 'Winter 2019-20'],
      dtype=object)

In [82]:
np.sort(air_quality['Time Period'].unique())

array(['2005', '2005-2007', '2009-2011', '2011', '2012-2014', '2013',
       '2015', '2015-2017', '2016', 'Annual Average 2009',
       'Annual Average 2010', 'Annual Average 2011',
       'Annual Average 2012', 'Annual Average 2013',
       'Annual Average 2014', 'Annual Average 2015',
       'Annual Average 2016', 'Annual Average 2017',
       'Annual Average 2018', 'Annual Average 2019',
       'Annual Average 2020', 'Summer 2009', 'Summer 2010', 'Summer 2011',
       'Summer 2012', 'Summer 2013', 'Summer 2014', 'Summer 2015',
       'Summer 2016', 'Summer 2017', 'Summer 2018', 'Summer 2019',
       'Summer 2020', 'Winter 2008-09', 'Winter 2009-10',
       'Winter 2010-11', 'Winter 2011-12', 'Winter 2012-13',
       'Winter 2013-14', 'Winter 2014-15', 'Winter 2015-16',
       'Winter 2016-17', 'Winter 2017-18', 'Winter 2018-19',
       'Winter 2019-20'], dtype=object)

In [75]:
air_quality['Time Period'].value_counts() / len(air_quality)

2009-2011              0.029773
2005-2007              0.029773
2012-2014              0.029773
2015-2017              0.029773
Summer 2010            0.026237
Summer 2019            0.026237
Summer 2009            0.026237
Summer 2013            0.026237
Summer 2014            0.026237
Summer 2015            0.026237
Summer 2016            0.026237
Summer 2017            0.026237
Summer 2018            0.026237
Winter 2008-09         0.026237
Summer 2020            0.026237
Summer 2012            0.026237
Winter 2009-10         0.026237
Winter 2010-11         0.026237
Winter 2011-12         0.026237
Winter 2012-13         0.026237
Winter 2013-14         0.026237
Winter 2014-15         0.026237
Winter 2015-16         0.026237
Summer 2011            0.026237
2005                   0.025245
2016                   0.019911
Annual Average 2020    0.017492
Annual Average 2011    0.017492
Winter 2018-19         0.017492
Winter 2017-18         0.017492
Winter 2016-17         0.017492
Annual A

In [17]:
air_quality['Start_Date'].unique()

array(['06/01/2013', '06/01/2014', '12/01/2008', '06/01/2009',
       '06/01/2010', '06/01/2011', '06/01/2012', '12/01/2009',
       '01/01/2005', '01/01/2013', '01/01/2009', '12/01/2010',
       '12/01/2011', '12/01/2012', '01/01/2015', '12/01/2013',
       '01/01/2011', '12/01/2014', '01/01/2016', '06/01/2015',
       '12/01/2015', '05/31/2016', '12/31/2015', '06/01/2017',
       '01/02/2012', '06/01/2018', '01/01/2017', '06/01/2019',
       '12/01/2016', '01/01/2018', '12/01/2017', '06/01/2020',
       '01/01/2019', '12/01/2018', '01/01/2020', '12/01/2019'],
      dtype=object)

In [18]:
air_quality['Start_Date'] = pd.to_datetime(air_quality['Start_Date'], infer_datetime_format=True)

In [19]:
air_quality['Start_Date'].min()

Timestamp('2005-01-01 00:00:00')

In [20]:
air_quality['Start_Date'].value_counts().sort_index() / len(air_quality)

2005-01-01    0.055018
2008-12-01    0.043729
2009-01-01    0.029773
2009-06-01    0.026237
2009-12-01    0.043729
2010-06-01    0.026237
2010-12-01    0.043729
2011-01-01    0.013274
2011-06-01    0.026237
2011-12-01    0.043729
2012-01-02    0.029773
2012-06-01    0.026237
2012-12-01    0.043729
2013-01-01    0.008932
2013-06-01    0.026237
2013-12-01    0.043729
2014-06-01    0.026237
2014-12-01    0.026237
2015-01-01    0.056197
2015-06-01    0.026237
2015-12-01    0.026237
2015-12-31    0.017492
2016-01-01    0.019911
2016-05-31    0.026237
2016-12-01    0.017492
2017-01-01    0.017492
2017-06-01    0.026237
2017-12-01    0.017492
2018-01-01    0.017492
2018-06-01    0.026237
2018-12-01    0.017492
2019-01-01    0.017492
2019-06-01    0.026237
2019-12-01    0.017492
2020-01-01    0.017492
2020-06-01    0.026237
Name: Start_Date, dtype: float64

In [21]:
air_quality.groupby('Start_Date')['Time Period'].value_counts()

Start_Date  Time Period        
2005-01-01  2005-2007              480
            2005                   407
2008-12-01  Winter 2008-09         423
            Annual Average 2009    282
2009-01-01  2009-2011              480
2009-06-01  Summer 2009            423
2009-12-01  Winter 2009-10         423
            Annual Average 2010    282
2010-06-01  Summer 2010            423
2010-12-01  Winter 2010-11         423
            Annual Average 2011    282
2011-01-01  2011                   214
2011-06-01  Summer 2011            423
2011-12-01  Winter 2011-12         423
            Annual Average 2012    282
2012-01-02  2012-2014              480
2012-06-01  Summer 2012            423
2012-12-01  Winter 2012-13         423
            Annual Average 2013    282
2013-01-01  2013                   144
2013-06-01  Summer 2013            423
2013-12-01  Winter 2013-14         423
            Annual Average 2014    282
2014-06-01  Summer 2014            423
2014-12-01  Winter 2014-15      

In [22]:
list(air_quality[air_quality['Time Period'].str.contains('Winter')]['Time Period'].unique())

['Winter 2008-09',
 'Winter 2009-10',
 'Winter 2010-11',
 'Winter 2011-12',
 'Winter 2012-13',
 'Winter 2013-14',
 'Winter 2014-15',
 'Winter 2015-16',
 'Winter 2016-17',
 'Winter 2017-18',
 'Winter 2018-19',
 'Winter 2019-20']

In [23]:
air_quality[air_quality['Time Period'].str.contains('Winter|Summer')].groupby('Time Period')['Start_Date'].value_counts()

Time Period     Start_Date
Summer 2009     2009-06-01    423
Summer 2010     2010-06-01    423
Summer 2011     2011-06-01    423
Summer 2012     2012-06-01    423
Summer 2013     2013-06-01    423
Summer 2014     2014-06-01    423
Summer 2015     2015-06-01    423
Summer 2016     2016-05-31    423
Summer 2017     2017-06-01    423
Summer 2018     2018-06-01    423
Summer 2019     2019-06-01    423
Summer 2020     2020-06-01    423
Winter 2008-09  2008-12-01    423
Winter 2009-10  2009-12-01    423
Winter 2010-11  2010-12-01    423
Winter 2011-12  2011-12-01    423
Winter 2012-13  2012-12-01    423
Winter 2013-14  2013-12-01    423
Winter 2014-15  2014-12-01    423
Winter 2015-16  2015-12-01    423
Winter 2016-17  2016-12-01    282
Winter 2017-18  2017-12-01    282
Winter 2018-19  2018-12-01    282
Winter 2019-20  2019-12-01    282
Name: Start_Date, dtype: int64

In [24]:
air_quality[air_quality['Time Period'].str.contains('Winter')]

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
4,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Morris Park and Bronxdale (CD11),Winter 2008-09,2008-12-01,5.89
5,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Williamsbridge and Baychester (CD12),Winter 2008-09,2008-12-01,5.75
8,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Greenpoint and Williamsburg (CD1),Winter 2008-09,2008-12-01,4.33
9,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Fort Greene and Brooklyn Heights (CD2),Winter 2008-09,2008-12-01,4.41
10,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Bedford Stuyvesant (CD3),Winter 2008-09,2008-12-01,4.73
...,...,...,...,...,...,...,...,...,...
15969,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,Queens Village (CD13),Winter 2019-20,2019-12-01,7.15
15972,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,Rockaway and Broad Channel (CD14),Winter 2019-20,2019-12-01,6.12
15975,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,St. George and Stapleton (CD1),Winter 2019-20,2019-12-01,7.43
15978,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,South Beach and Willowbrook (CD2),Winter 2019-20,2019-12-01,6.89


In [25]:
air_quality.sample(10)

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
4028,375,Nitrogen Dioxide (NO2),Mean,ppb,UHF34,Long Island City - Astoria,Summer 2011,2011-06-01,20.69
15792,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,UHF42,Canarsie - Flatlands,Summer 2020,2020-06-01,6.60
2711,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,Lower East Side and Chinatown (CD3),Winter 2008-09,2008-12-01,14.24
2041,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Flatlands and Canarsie (CD18),Winter 2011-12,2011-12-01,20.27
6550,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Fordham - Bronx Pk,2015,2015-01-01,16.60
0,386,Ozone (O3),Mean,ppb,CD,Coney Island (CD13),Summer 2013,2013-06-01,34.64
8895,644,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Rockaways,2016,2016-01-01,6.50
1846,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Greenpoint and Williamsburg (CD1),Winter 2008-09,2008-12-01,32.58
10423,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,UHF34,Upper West Side,Summer 2016,2016-05-31,9.12
1547,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Greenwich Village and Soho (CD2),Winter 2008-09,2008-12-01,8.68


In [26]:
air_quality[air_quality['Geo Type Name'].str.contains('Borough')]['Time Period'].unique()

array(['Summer 2013', 'Summer 2014', 'Summer 2009', 'Summer 2010',
       'Summer 2011', 'Summer 2012', '2013', '2005', '2005-2007',
       '2009-2011', 'Winter 2008-09', 'Winter 2009-10', 'Winter 2010-11',
       'Winter 2011-12', 'Winter 2012-13', 'Annual Average 2009',
       'Annual Average 2010', 'Annual Average 2011',
       'Annual Average 2012', 'Annual Average 2013', '2015',
       'Winter 2013-14', 'Annual Average 2014', '2011', '2016',
       'Annual Average 2015', 'Summer 2015', 'Winter 2014-15',
       'Winter 2015-16', 'Summer 2016', 'Annual Average 2016',
       '2012-2014', 'Summer 2017', 'Annual Average 2017',
       'Winter 2016-17', 'Annual Average 2018', 'Summer 2018',
       'Winter 2017-18', '2015-2017', 'Annual Average 2019',
       'Summer 2019', 'Winter 2018-19', 'Annual Average 2020',
       'Winter 2019-20', 'Summer 2020'], dtype=object)

In [27]:
air_quality[air_quality['Geo Type Name'].str.contains('Borough')].groupby('Name')['Geo Place Name'].value_counts()

Name                                                       Geo Place Name
Air Toxics Concentrations- Average Benzene Concentrations  Bronx             2
                                                           Brooklyn          2
                                                           Manhattan         2
                                                           Queens            2
                                                           Staten Island     2
                                                                            ..
Traffic Density- Annual Vehicle Miles Traveled for Trucks  Bronx             1
                                                           Brooklyn          1
                                                           Manhattan         1
                                                           Queens            1
                                                           Staten Island     1
Name: Geo Place Name, Length: 95, dtype: int64

In [28]:
list(air_quality['Name'].unique())

['Ozone (O3)',
 'Sulfur Dioxide (SO2)',
 'PM2.5-Attributable Deaths',
 'Boiler Emissions- Total SO2 Emissions',
 'Boiler Emissions- Total PM2.5 Emissions',
 'Boiler Emissions- Total NOx Emissions',
 'Air Toxics Concentrations- Average Benzene Concentrations',
 'Air Toxics Concentrations- Average Formaldehyde Concentrations',
 'PM2.5-Attributable Asthma Emergency Department Visits',
 'PM2.5-Attributable Respiratory Hospitalizations (Adults 20 Yrs and Older)',
 'PM2.5-Attributable Cardiovascular Hospitalizations (Adults 40 Yrs and Older)',
 'Traffic Density- Annual Vehicle Miles Traveled',
 'O3-Attributable Cardiac and Respiratory Deaths',
 'O3-Attributable Asthma Emergency Department Visits',
 'O3-Attributable Asthma Hospitalizations',
 'Traffic Density- Annual Vehicle Miles Traveled for Cars',
 'Traffic Density- Annual Vehicle Miles Traveled for Trucks',
 'Nitrogen Dioxide (NO2)',
 'Fine Particulate Matter (PM2.5)']

In [29]:
air_quality.groupby(['Name', 'Start_Date'])['Time Period'].value_counts()

Name                                                            Start_Date  Time Period
Air Toxics Concentrations- Average Benzene Concentrations       2005-01-01  2005            48
                                                                2011-01-01  2011           107
Air Toxics Concentrations- Average Formaldehyde Concentrations  2005-01-01  2005            48
                                                                2011-01-01  2011           107
Boiler Emissions- Total NOx Emissions                           2013-01-01  2013            48
                                                                                          ... 
Traffic Density- Annual Vehicle Miles Traveled                  2016-01-01  2016           107
Traffic Density- Annual Vehicle Miles Traveled for Cars         2005-01-01  2005           107
                                                                2016-01-01  2016           107
Traffic Density- Annual Vehicle Miles Traveled for Trucks

In [69]:
air_quality['Geo Type Name'].unique()

array(['CD', 'Borough', 'UHF42', 'Citywide', 'UHF34'], dtype=object)

In [71]:
air_quality[air_quality['Geo Type Name'].str.contains('CD')]

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
0,386,Ozone (O3),Mean,ppb,CD,Coney Island (CD13),Summer 2013,2013-06-01,34.64
1,386,Ozone (O3),Mean,ppb,CD,Coney Island (CD13),Summer 2014,2014-06-01,33.22
4,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Morris Park and Bronxdale (CD11),Winter 2008-09,2008-12-01,5.89
5,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Williamsbridge and Baychester (CD12),Winter 2008-09,2008-12-01,5.75
8,383,Sulfur Dioxide (SO2),Mean,ppb,CD,Greenpoint and Williamsburg (CD1),Winter 2008-09,2008-12-01,4.33
...,...,...,...,...,...,...,...,...,...
16117,386,Ozone (O3),Mean,ppb,CD,Park Slope and Carroll Gardens (CD6),Summer 2020,2020-06-01,28.70
16118,386,Ozone (O3),Mean,ppb,CD,East New York and Starrett City (CD5),Summer 2020,2020-06-01,29.56
16119,386,Ozone (O3),Mean,ppb,CD,Bushwick (CD4),Summer 2020,2020-06-01,29.65
16120,386,Ozone (O3),Mean,ppb,CD,Bedford Stuyvesant (CD3),Summer 2020,2020-06-01,29.28


In [72]:
air_quality[air_quality['Geo Type Name'].str.contains('UHF42')]

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
22,639,PM2.5-Attributable Deaths,Estimated Annual Rate - Adults 30 Yrs and Older,"per 100,000 adults",UHF42,Kingsbridge - Riverdale,2005-2007,2005-01-01,117.70
23,639,PM2.5-Attributable Deaths,Estimated Annual Rate - Adults 30 Yrs and Older,"per 100,000 adults",UHF42,Northeast Bronx,2005-2007,2005-01-01,77.30
24,639,PM2.5-Attributable Deaths,Estimated Annual Rate - Adults 30 Yrs and Older,"per 100,000 adults",UHF42,Fordham - Bronx Pk,2005-2007,2005-01-01,67.30
25,639,PM2.5-Attributable Deaths,Estimated Annual Rate - Adults 30 Yrs and Older,"per 100,000 adults",UHF42,Pelham - Throgs Neck,2005-2007,2005-01-01,73.60
26,639,PM2.5-Attributable Deaths,Estimated Annual Rate - Adults 30 Yrs and Older,"per 100,000 adults",UHF42,Crotona -Tremont,2005-2007,2005-01-01,65.80
...,...,...,...,...,...,...,...,...,...
16078,386,Ozone (O3),Mean,ppb,UHF42,Crotona -Tremont,Summer 2020,2020-06-01,30.15
16079,386,Ozone (O3),Mean,ppb,UHF42,Pelham - Throgs Neck,Summer 2020,2020-06-01,32.05
16080,386,Ozone (O3),Mean,ppb,UHF42,Fordham - Bronx Pk,Summer 2020,2020-06-01,30.17
16081,386,Ozone (O3),Mean,ppb,UHF42,Northeast Bronx,Summer 2020,2020-06-01,30.85


In [73]:
air_quality[air_quality['Geo Type Name'].str.contains('UHF34')]

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
3600,383,Sulfur Dioxide (SO2),Mean,ppb,UHF34,Kingsbridge - Riverdale,Winter 2008-09,2008-12-01,6.62
3601,383,Sulfur Dioxide (SO2),Mean,ppb,UHF34,Northeast Bronx,Winter 2008-09,2008-12-01,5.38
3602,383,Sulfur Dioxide (SO2),Mean,ppb,UHF34,Fordham - Bronx Pk,Winter 2008-09,2008-12-01,9.48
3603,383,Sulfur Dioxide (SO2),Mean,ppb,UHF34,Pelham - Throgs Neck,Winter 2008-09,2008-12-01,5.15
3604,383,Sulfur Dioxide (SO2),Mean,ppb,UHF34,Greenpoint,Winter 2008-09,2008-12-01,4.25
...,...,...,...,...,...,...,...,...,...
16036,386,Ozone (O3),Mean,ppb,UHF34,Bedford Stuyvesant - Crown Heights,Summer 2020,2020-06-01,29.16
16037,386,Ozone (O3),Mean,ppb,UHF34,Downtown - Heights - Slope,Summer 2020,2020-06-01,28.79
16038,386,Ozone (O3),Mean,ppb,UHF34,Greenpoint,Summer 2020,2020-06-01,29.71
16039,386,Ozone (O3),Mean,ppb,UHF34,Pelham - Throgs Neck,Summer 2020,2020-06-01,32.05


In [30]:
air_quality_boros = air_quality[air_quality['Geo Type Name'].str.contains('Borough')]
air_quality_boros = air_quality_boros.drop(['Geo Type Name'], axis=1)

In [31]:
air_quality_boros.sample(10)

,Indicator ID,Name,Measure,Measure Info,Geo Place Name,Time Period,Start_Date,Data Value
15842,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Queens,Winter 2019-20,2019-12-01,7.55
9389,375,Nitrogen Dioxide (NO2),Mean,ppb,Manhattan,Annual Average 2015,2015-01-01,25.18
8970,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Queens,Summer 2015,2015-06-01,9.21
6420,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Staten Island,Winter 2012-13,2012-12-01,9.72
6341,375,Nitrogen Dioxide (NO2),Mean,ppb,Bronx,Winter 2012-13,2012-12-01,24.03
15137,375,Nitrogen Dioxide (NO2),Mean,ppb,Bronx,Annual Average 2020,2020-01-01,17.10
3251,386,Ozone (O3),Mean,ppb,Queens,Summer 2013,2013-06-01,31.60
796,657,PM2.5-Attributable Asthma Emergency Department...,Estimated Annual Rate- 18 Yrs and Older,"per 100,000 adults",Brooklyn,2009-2011,2009-01-01,48.50
6397,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Brooklyn,Winter 2008-09,2008-12-01,12.88
506,652,O3-Attributable Cardiac and Respiratory Deaths,Estimated Annual Rate,"per 100,000 residents",Staten Island,2005-2007,2005-01-01,7.80


In [32]:
for time_period in air_quality_boros['Time Period'].sort_values().unique():
    print(time_period, 'length:', len(time_period))

2005 length: 4
2005-2007 length: 9
2009-2011 length: 9
2011 length: 4
2012-2014 length: 9
2013 length: 4
2015 length: 4
2015-2017 length: 9
2016 length: 4
Annual Average 2009 length: 19
Annual Average 2010 length: 19
Annual Average 2011 length: 19
Annual Average 2012 length: 19
Annual Average 2013 length: 19
Annual Average 2014 length: 19
Annual Average 2015 length: 19
Annual Average 2016 length: 19
Annual Average 2017 length: 19
Annual Average 2018 length: 19
Annual Average 2019 length: 19
Annual Average 2020 length: 19
Summer 2009 length: 11
Summer 2010 length: 11
Summer 2011 length: 11
Summer 2012 length: 11
Summer 2013 length: 11
Summer 2014 length: 11
Summer 2015 length: 11
Summer 2016 length: 11
Summer 2017 length: 11
Summer 2018 length: 11
Summer 2019 length: 11
Summer 2020 length: 11
Winter 2008-09 length: 14
Winter 2009-10 length: 14
Winter 2010-11 length: 14
Winter 2011-12 length: 14
Winter 2012-13 length: 14
Winter 2013-14 length: 14
Winter 2014-15 length: 14
Winter 2015-16 

In [33]:
air_quality.groupby('Time Period')['Start_Date'].value_counts()

Time Period          Start_Date
2005                 2005-01-01    407
2005-2007            2005-01-01    480
2009-2011            2009-01-01    480
2011                 2011-01-01    214
2012-2014            2012-01-02    480
2013                 2013-01-01    144
2015                 2015-01-01    144
2015-2017            2015-01-01    480
2016                 2016-01-01    321
Annual Average 2009  2008-12-01    282
Annual Average 2010  2009-12-01    282
Annual Average 2011  2010-12-01    282
Annual Average 2012  2011-12-01    282
Annual Average 2013  2012-12-01    282
Annual Average 2014  2013-12-01    282
Annual Average 2015  2015-01-01    282
Annual Average 2016  2015-12-31    282
Annual Average 2017  2017-01-01    282
Annual Average 2018  2018-01-01    282
Annual Average 2019  2019-01-01    282
Annual Average 2020  2020-01-01    282
Summer 2009          2009-06-01    423
Summer 2010          2010-06-01    423
Summer 2011          2011-06-01    423
Summer 2012          2012-06-01 

In [34]:
def create_end_dates(air_quality_boros_dict):
    end_dates = []
    for row in air_quality_boros_dict:
        time_period_str_len = len(row['Time Period'])
        year = row['Start_Date'].year
        if time_period_str_len == 4:
            date = datetime.date(year, 12, 31)
        elif time_period_str_len == 9:
            date = datetime.date(year + 2, 12, 31)
        elif time_period_str_len == 11:
            date = datetime.date(year, 8, 31)
        elif time_period_str_len == 14:
            date = datetime.date(year + 1, 2, 28)
        elif time_period_str_len == 19:
            year = int(row['Time Period'][-4:])
            date = datetime.date(year, 12, 31)
        end_dates.append(date)
    return end_dates

In [35]:
air_quality_boros['end_date'] = create_end_dates(air_quality_boros.to_dict('records'))

In [36]:
air_quality_boros.sample(10)

,Indicator ID,Name,Measure,Measure Info,Geo Place Name,Time Period,Start_Date,Data Value,end_date
6411,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Bronx,Winter 2011-12,2011-12-01,9.93,2012-02-28
12274,375,Nitrogen Dioxide (NO2),Mean,ppb,Queens,Winter 2016-17,2016-12-01,24.83,2017-02-28
13874,651,PM2.5-Attributable Cardiovascular Hospitalizat...,Estimated Annual Rate,"per 100,000 adults",Manhattan,2015-2017,2015-01-01,14.60,2017-12-31
847,648,PM2.5-Attributable Asthma Emergency Department...,Estimated Annual Rate- Children 0 to 17 Yrs Old,"per 100,000 children",Staten Island,2009-2011,2009-01-01,50.50,2011-12-31
6302,383,Sulfur Dioxide (SO2),Mean,ppb,Brooklyn,Winter 2009-10,2009-12-01,2.66,2010-02-28
15148,375,Nitrogen Dioxide (NO2),Mean,ppb,Queens,Summer 2020,2020-06-01,10.00,2020-08-31
3,386,Ozone (O3),Mean,ppb,Bronx,Summer 2014,2014-06-01,31.15,2014-08-31
13253,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Brooklyn,Annual Average 2018,2018-01-01,7.19,2018-12-31
6358,375,Nitrogen Dioxide (NO2),Mean,ppb,Manhattan,Summer 2011,2011-06-01,27.72,2011-08-31
6465,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Staten Island,Annual Average 2012,2011-12-01,8.35,2012-12-31


In [37]:
air_quality_boros['Indicator ID'].nunique()

22

In [64]:
print(air_quality.groupby(['Name', 'Measure'])['Indicator ID'].value_counts())
print(len(air_quality.groupby(['Name', 'Measure'])['Indicator ID'].value_counts()))

Name                                                                          Measure                                          Indicator ID
Air Toxics Concentrations- Average Benzene Concentrations                     Annual Average Concentration                     646              155
Air Toxics Concentrations- Average Formaldehyde Concentrations                Annual Average Concentration                     647              155
Boiler Emissions- Total NOx Emissions                                         Number per km2                                   642               96
Boiler Emissions- Total PM2.5 Emissions                                       Number per km2                                   641               96
Boiler Emissions- Total SO2 Emissions                                         Number per km2                                   640               96
Fine Particulate Matter (PM2.5)                                               Mean                                      

In [39]:
air_quality_boros = air_quality_boros.drop(['Indicator ID'], axis=1)

In [40]:
air_quality_boros.sample(10)

,Name,Measure,Measure Info,Geo Place Name,Time Period,Start_Date,Data Value,end_date
6430,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Staten Island,Summer 2010,2010-06-01,11.43,2010-08-31
14172,Nitrogen Dioxide (NO2),Mean,ppb,Queens,Summer 2019,2019-06-01,13.06,2019-08-31
6380,Nitrogen Dioxide (NO2),Mean,ppb,Staten Island,Annual Average 2010,2009-12-01,15.35,2010-12-31
6330,Nitrogen Dioxide (NO2),Mean,ppb,Staten Island,Winter 2009-10,2009-12-01,19.14,2010-02-28
9392,Nitrogen Dioxide (NO2),Mean,ppb,Queens,Annual Average 2015,2015-01-01,18.51,2015-12-31
15844,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Manhattan,Summer 2020,2020-06-01,7.80,2020-08-31
407,PM2.5-Attributable Respiratory Hospitalization...,Estimated Annual Rate,"per 100,000 adults",Manhattan,2005-2007,2005-01-01,18.30,2007-12-31
6442,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Brooklyn,Summer 2013,2013-06-01,10.27,2013-08-31
15150,Nitrogen Dioxide (NO2),Mean,ppb,Staten Island,Winter 2019-20,2019-12-01,17.87,2020-02-28
6434,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,Queens,Summer 2011,2011-06-01,11.23,2011-08-31


In [67]:
air_quality_boros.groupby(['Name'])['Measure'].value_counts()

Name                                                                          Measure                                        
Air Toxics Concentrations- Average Benzene Concentrations                     Annual Average Concentration                        10
Air Toxics Concentrations- Average Formaldehyde Concentrations                Annual Average Concentration                        10
Boiler Emissions- Total NOx Emissions                                         Number per km2                                      10
Boiler Emissions- Total PM2.5 Emissions                                       Number per km2                                      10
Boiler Emissions- Total SO2 Emissions                                         Number per km2                                      10
Fine Particulate Matter (PM2.5)                                               Mean                                               180
Nitrogen Dioxide (NO2)                                                      

In [84]:
air_quality_boros[air_quality_boros['Time Period'].str.contains("Annual")]['Measure'].unique()

array(['Mean'], dtype=object)

In [86]:
air_quality_boros[air_quality_boros['Time Period'].str.contains("Annual")]['Name'].unique()

array(['Nitrogen Dioxide (NO2)', 'Fine Particulate Matter (PM2.5)'],
      dtype=object)

In [88]:
air_quality_boros[air_quality_boros['Measure'].str.contains("Mean")]['Time Period'].unique()

array(['Summer 2013', 'Summer 2014', 'Summer 2009', 'Summer 2010',
       'Summer 2011', 'Summer 2012', 'Winter 2008-09', 'Winter 2009-10',
       'Winter 2010-11', 'Winter 2011-12', 'Winter 2012-13',
       'Annual Average 2009', 'Annual Average 2010',
       'Annual Average 2011', 'Annual Average 2012',
       'Annual Average 2013', 'Winter 2013-14', 'Annual Average 2014',
       'Annual Average 2015', 'Summer 2015', 'Winter 2014-15',
       'Winter 2015-16', 'Summer 2016', 'Annual Average 2016',
       'Summer 2017', 'Annual Average 2017', 'Winter 2016-17',
       'Annual Average 2018', 'Summer 2018', 'Winter 2017-18',
       'Annual Average 2019', 'Summer 2019', 'Winter 2018-19',
       'Annual Average 2020', 'Winter 2019-20', 'Summer 2020'],
      dtype=object)

## Traffic Volume Dataset Cleaning

In [41]:
traffic_volume.sample(10)

,RequestID,Boro,Yr,M,D,HH,MM,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction
5511352,12081,Bronx,2010,9,30,1,0,3,79581,POINT (1010420.5 251153.9),JEROME AVE BET CAMERON PL & 181ST ST SR,E 181 ST/W 181 ST,CAMERON PL,SB
25181347,14402,Brooklyn,2013,5,1,1,45,0,26671,POINT (991231.3 166952.1),E 2 ST,AV J,AV I,NB
14514328,28831,Bronx,2019,1,7,4,30,23,71482,POINT (1004563.600853212 246371.34678012744),UNDERCLIFF AVENUE,High Bridge,West 171 Street,SB
21850547,1297,Queens,2011,8,31,4,45,27,129615,POINT (1007236.7 200885.7),GRAND AV,57 ST,PAGE PL,WB
20016211,10525,Bronx,2012,10,14,22,0,43,78136,POINT (1011190.1 242400.6),E 169 ST,FULTON AV,FRANKLIN AV,WB
884820,11564,Brooklyn,2012,10,22,14,45,222,25655,POINT (994948.4 158601.3),CONEY ISLAND AV,HOMECREST CT,HUTCHINSON CT,SB
2640344,19657,Queens,2015,5,10,20,30,130,157151,POINT (1066742.1952774417 213469.14231069732),UNION TURNPIKE,Dead End,Hewlett Street,WB
13081291,28287,Brooklyn,2018,5,30,13,15,10,253118,POINT (990772.3095365886 175807.6977522518),FT HAMILTON PARKWAY,Culver/ 6 Avenue Line,East 5 Street,WB
6482060,24564,Brooklyn,2016,10,18,8,45,44,25938,POINT (996885.4222728872 153311.1680553053),SHEEPSHEAD BAY ROAD,East 14 Street,East 15 Street,EB
5093134,28831,Bronx,2019,1,16,21,30,104,147036,POINT (1022204.8146712111 242900.76316457777),WHITE PLAINS ROAD,Dead End,Cross Bronx Expressway Sr South,SB


In [42]:
traffic_volume.shape

(27190511, 14)

In [43]:
print(traffic_volume.isnull().sum() / len(traffic_volume))

RequestID    0.000000
Boro         0.000000
Yr           0.000000
M            0.000000
D            0.000000
HH           0.000000
MM           0.000000
Vol          0.000000
SegmentID    0.000000
WktGeom      0.000000
street       0.000000
fromSt       0.000000
toSt         0.000074
Direction    0.000000
dtype: float64


In [44]:
print(traffic_volume.nunique() / len(traffic_volume))

RequestID    2.607527e-04
Boro         1.838877e-07
Yr           5.884406e-07
M            4.413304e-07
D            1.140104e-06
HH           8.826609e-07
MM           1.471101e-07
Vol          1.476986e-04
SegmentID    5.499345e-04
WktGeom      7.525787e-04
street       2.482484e-04
fromSt       2.361486e-04
toSt         2.175391e-04
Direction    2.206652e-07
dtype: float64


In [45]:
traffic_volume.nunique()

RequestID     7090
Boro             5
Yr              16
M               12
D               31
HH              24
MM               4
Vol           4016
SegmentID    14953
WktGeom      20463
street        6750
fromSt        6421
toSt          5915
Direction        6
dtype: int64

In [46]:
traffic_volume.dtypes

RequestID     int64
Boro         object
Yr            int64
M             int64
D             int64
HH            int64
MM            int64
Vol           int64
SegmentID     int64
WktGeom      object
street       object
fromSt       object
toSt         object
Direction    object
dtype: object

In [47]:
traffic_volume.Yr.min()

2000

In [48]:
traffic_volume = traffic_volume[traffic_volume['Yr'] >= 2005]

In [49]:
traffic_volume.shape

(27188607, 14)

In [50]:
traffic_volume['Yr'].value_counts().sort_index()

2006        664
2007      11780
2008      68591
2009    1012766
2010    1421397
2011    1238391
2012    2434583
2013    2829656
2014    3708367
2015    3232005
2016    3362243
2017    3013530
2018    2046443
2019    2365633
2020     442558
Name: Yr, dtype: int64

In [51]:
traffic_volume = traffic_volume[traffic_volume['Yr'] > 2008]

In [52]:
traffic_volume.shape

(27107572, 14)

In [53]:
traffic_volume['date_time'] = pd.to_datetime(dict(year=traffic_volume.Yr, \
                                                  month=traffic_volume.M, \
                                                  day=traffic_volume.D, \
                                                  hour=traffic_volume.HH, \
                                                  minute=traffic_volume.MM))

In [54]:
traffic_volume = traffic_volume.drop(['Yr', 'M', 'D', 'HH', 'MM'], axis=1)

In [55]:
traffic_volume.sample(10)

,RequestID,Boro,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,date_time
21452560,12038,Brooklyn,20,41720,POINT (1000272.5 172937.1),AV D,BROOKLYN AV,E 37 ST,EB,2013-02-02 02:45:00
20511337,30153,Queens,519,64059,POINT (1059473.7342565234 181796.41435752026),SOUTH CONDUIT AVENUE,Hook Creek Boulevard,Dead end,EB,2019-03-27 12:00:00
14599670,23837,Bronx,330,9008826,POINT (1021943.9345069327 251657.43481426392),PELHAM PARKWAY,Dead End,Dead end,WB,2016-10-12 17:00:00
18574636,11575,Manhattan,232,33547,POINT (984203.5 211823.7),9 AV,W 25 ST,W 26 ST,SB,2012-10-04 05:45:00
23608944,28337,Queens,192,136512,POINT (1000514.6120462302 208587.39315177445),LONG ISLAND EXPRESSWAY,Dutch Kills Shoreline,Borden Avenue,EB,2018-06-14 19:00:00
23690646,23133,Queens,2,101100,POINT (1050255.524319251 216761.28283362705),220 STREET,46 Avenue,Northern Boulevard,NB,2016-03-21 22:15:00
22851242,18083,Bronx,97,88518,POINT (1026942.2982799585 258345.73751860857),EASTCHESTER ROAD,Givan Avenue,Tillotson Avenue,SB,2014-09-18 15:00:00
25125732,24311,Queens,13,96375,POINT (1043452.3764165054 208567.90037198758),64 AVENUE,186 Lane,188 Street,EB,2016-09-15 06:45:00
6105869,3092,Manhattan,70,173215,POINT (1002365.9 236956.7),S/B 5 AVE @ W 142 ST,W 142 ST/HARLEM RIVER DR SR W,BEND,SB,2009-10-05 18:30:00
14416981,3151,Manhattan,0,34006,POINT (988525.8 210901.9),N/B MADISON AVE @ E 31 ST,E 30 ST,E 31 ST,NB,2009-02-06 21:00:00


In [56]:
traffic_volume['date_time'].dt.year.value_counts().sort_index()

2009    1012766
2010    1421397
2011    1238391
2012    2434583
2013    2829656
2014    3708367
2015    3232005
2016    3362243
2017    3013530
2018    2046443
2019    2365633
2020     442558
Name: date_time, dtype: int64

In [96]:
traffic_volume['date_time'].dt.year.value_counts().sort_index() / len(traffic_volume)

2009    0.037361
2010    0.052435
2011    0.045684
2012    0.089812
2013    0.104386
2014    0.136802
2015    0.119229
2016    0.124033
2017    0.111169
2018    0.075493
2019    0.087268
2020    0.016326
Name: date_time, dtype: float64

In [97]:
traffic_volume.head()

,RequestID,Boro,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,date_time
25543663,32417,Queens,20,67665,POINT (1004228.4823799994 215767.68782613552),31 STREET,34 Avenue,35 Avenue,SB,2020-11-22 23:45:00
20968423,32417,Queens,3,101621,POINT (1055667.922729934 216597.78334720692),DOUGLASTON PARKWAY,Maryland Road,Van Zandt Avenue,SB,2020-11-22 23:45:00
6368254,32417,Queens,27,148877,POINT (1015917.5752772316 218664.23589469263),23 AVENUE,Dead End,85 Street,WB,2020-11-22 23:45:00
7417214,32417,Queens,22,155846,POINT (1050166.9282559236 199291.28045636677),JAMAICA AVENUE,197 Street,198 Street,WB,2020-11-22 23:45:00
18926986,32417,Queens,5,76510,POINT (1017222.1345317845 216101.64900761063),31 AVENUE,87 Street,88 Street,WB,2020-11-22 23:45:00


In [57]:
traffic_volume.sort_values(["date_time"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True);
traffic_volume.head(15)

,RequestID,Boro,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,date_time
25543663,32417,Queens,20,67665,POINT (1004228.4823799994 215767.68782613552),31 STREET,34 Avenue,35 Avenue,SB,2020-11-22 23:45:00
20968423,32417,Queens,3,101621,POINT (1055667.922729934 216597.78334720692),DOUGLASTON PARKWAY,Maryland Road,Van Zandt Avenue,SB,2020-11-22 23:45:00
6368254,32417,Queens,27,148877,POINT (1015917.5752772316 218664.23589469263),23 AVENUE,Dead End,85 Street,WB,2020-11-22 23:45:00
7417214,32417,Queens,22,155846,POINT (1050166.9282559236 199291.28045636677),JAMAICA AVENUE,197 Street,198 Street,WB,2020-11-22 23:45:00
18926986,32417,Queens,5,76510,POINT (1017222.1345317845 216101.64900761063),31 AVENUE,87 Street,88 Street,WB,2020-11-22 23:45:00
13407547,32417,Queens,60,155773,POINT (1027092.2981505651 190399.9018293268),ATLANTIC AVENUE,97 Street,98 Street,WB,2020-11-22 23:45:00
7423333,32417,Queens,8,75313,POINT (1019421.0343045937 208800.348152701),51 AVENUE,90 Street,92 Street,EB,2020-11-22 23:45:00
18674125,32417,Queens,33,45497,POINT (1009925.0151697205 199046.51050050836),METROPOLITAN AVENUE,55 Street,56 Street,WB,2020-11-22 23:45:00
18956053,32417,Queens,6,145416,POINT (1046510.2750956557 205583.03286028266),UNION TURNPIKE,Dead End,Dead end,EB,2020-11-22 23:45:00
9171262,32417,Queens,5,101855,POINT (1057510.6819890984 218109.6099654521),MARATHON PARKWAY,Rushmore Avenue,Morenci Lane,SB,2020-11-22 23:45:00


In [141]:
traffic_vol_daily = traffic_volume.groupby(['Boro', traffic_volume['date_time'].dt.date])['Vol'].mean().reset_index()
traffic_vol_daily.columns = traffic_vol_daily.columns.str.lower()
traffic_vol_daily.rename(columns={'date_time':'date'}, inplace=True)

In [142]:
traffic_vol_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12737 entries, 0 to 12736
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   boro    12737 non-null  object 
 1   date    12737 non-null  object 
 2   vol     12737 non-null  float64
dtypes: float64(1), object(2)
memory usage: 298.6+ KB


In [143]:
print(len(traffic_vol_daily[traffic_vol_daily['boro'] == 'Bronx']))
traffic_vol_daily[traffic_vol_daily['boro'] == 'Bronx'].head(10)

2306


,boro,date,vol
0,Bronx,2009-01-20,55.559748
1,Bronx,2009-01-21,42.722222
2,Bronx,2009-01-22,45.707224
3,Bronx,2009-01-23,49.243056
4,Bronx,2009-01-24,33.958333
5,Bronx,2009-01-25,25.475694
6,Bronx,2009-01-26,2.755556
7,Bronx,2009-02-21,57.924731
8,Bronx,2009-02-22,32.234375
9,Bronx,2009-02-23,42.192708


In [144]:
print(len(traffic_vol_daily[traffic_vol_daily['boro'] == 'Queens']))
traffic_vol_daily[traffic_vol_daily['boro'] == 'Queens'].head(10)

3098


,boro,date,vol
7970,Queens,2009-01-12,13.409091
7971,Queens,2009-01-13,89.291498
7972,Queens,2009-01-14,66.995444
7973,Queens,2009-01-15,64.712500
7974,Queens,2009-01-16,87.118483
7975,Queens,2009-01-17,8.885057
7976,Queens,2009-02-09,34.076517
7977,Queens,2009-02-10,24.819940
7978,Queens,2009-02-11,25.796131
7979,Queens,2009-02-12,26.970238


In [145]:
print(traffic_vol_daily[traffic_vol_daily['boro'] == 'Brooklyn'].shape[0])
traffic_vol_daily[traffic_vol_daily['boro'] == 'Brooklyn'].head(10)

3143


,boro,date,vol
2306,Brooklyn,2009-01-12,40.478992
2307,Brooklyn,2009-01-13,33.807292
2308,Brooklyn,2009-01-14,18.541667
2309,Brooklyn,2009-01-15,12.307292
2310,Brooklyn,2009-01-16,34.046875
2311,Brooklyn,2009-01-17,29.776042
2312,Brooklyn,2009-01-18,22.140625
2313,Brooklyn,2009-01-19,17.057292
2314,Brooklyn,2009-01-20,34.370052
2315,Brooklyn,2009-01-21,55.906667


In [146]:
print(traffic_vol_daily[traffic_vol_daily['boro'] == 'Manhattan'].shape[0])
traffic_vol_daily[traffic_vol_daily['boro'] == 'Manhattan'].head(10)

2521


,boro,date,vol
5449,Manhattan,2009-01-08,54.047619
5450,Manhattan,2009-01-09,88.436326
5451,Manhattan,2009-01-10,70.918750
5452,Manhattan,2009-01-11,59.256250
5453,Manhattan,2009-01-12,71.082038
5454,Manhattan,2009-01-13,65.396577
5455,Manhattan,2009-01-14,69.708333
5456,Manhattan,2009-01-15,69.952381
5457,Manhattan,2009-01-16,79.558780
5458,Manhattan,2009-01-17,71.813244


In [148]:
print(traffic_vol_daily[traffic_vol_daily['boro'] == 'Staten Island'].shape[0])
traffic_vol_daily[traffic_vol_daily['boro'] == 'Staten Island'].head(10)

1669


,boro,date,vol
11068,Staten Island,2009-02-06,73.077844
11069,Staten Island,2009-02-07,46.930556
11070,Staten Island,2009-02-08,38.333333
11071,Staten Island,2009-02-09,42.625000
11072,Staten Island,2009-02-10,30.187500
11073,Staten Island,2009-02-11,40.204861
11074,Staten Island,2009-02-12,24.638889
11075,Staten Island,2009-02-13,5.555556
11076,Staten Island,2009-02-14,3.614583
11077,Staten Island,2009-02-15,2.788194


## EPA data

In [ ]:
email = 'daguila000@citymail.cuny.edu'
key = 'cobaltcrane81'
param = ''
bdate = ''
edate = ''
state = '36'
counties = ['']


In [154]:
import json

In [158]:
url = 'https://aqs.epa.gov/data/api/list/states?email=test@aqs.api&key=test'

response = requests.get(url)
j = json.loads(response.text)
print(json.dumps(j, indent=2))

{
  "Header": [
    {
      "status": "Success",
      "request_time": "2022-12-14T01:39:48-05:00",
      "url": "https://aqs.epa.gov/data/api/list/states?email=test@aqs.api&key=test",
      "rows": 56
    }
  ],
  "Data": [
    {
      "code": "01",
      "value_represented": "Alabama"
    },
    {
      "code": "02",
      "value_represented": "Alaska"
    },
    {
      "code": "04",
      "value_represented": "Arizona"
    },
    {
      "code": "05",
      "value_represented": "Arkansas"
    },
    {
      "code": "06",
      "value_represented": "California"
    },
    {
      "code": "08",
      "value_represented": "Colorado"
    },
    {
      "code": "09",
      "value_represented": "Connecticut"
    },
    {
      "code": "10",
      "value_represented": "Delaware"
    },
    {
      "code": "11",
      "value_represented": "District Of Columbia"
    },
    {
      "code": "12",
      "value_represented": "Florida"
    },
    {
      "code": "13",
      "value_represented": 

In [166]:
#print(j['Data']['value_represented'])

state_code = ''

for state in j['Data']:
    if state['value_represented'] == 'New York':
        state_code = state['code']
        
print(state_code)

36


In [167]:
url = f'https://aqs.epa.gov/data/api/list/countiesByState?email=test@aqs.api&key=test&state={state_code}'

response = requests.get(url)
j = json.loads(response.text)
print(json.dumps(j, indent=2))

{
  "Header": [
    {
      "status": "Success",
      "request_time": "2022-12-14T01:47:35-05:00",
      "url": "https://aqs.epa.gov/data/api/list/countiesByState?email=test@aqs.api&key=test&state=36",
      "rows": 62
    }
  ],
  "Data": [
    {
      "code": "001",
      "value_represented": "Albany"
    },
    {
      "code": "003",
      "value_represented": "Allegany"
    },
    {
      "code": "005",
      "value_represented": "Bronx"
    },
    {
      "code": "007",
      "value_represented": "Broome"
    },
    {
      "code": "009",
      "value_represented": "Cattaraugus"
    },
    {
      "code": "011",
      "value_represented": "Cayuga"
    },
    {
      "code": "013",
      "value_represented": "Chautauqua"
    },
    {
      "code": "015",
      "value_represented": "Chemung"
    },
    {
      "code": "017",
      "value_represented": "Chenango"
    },
    {
      "code": "019",
      "value_represented": "Clinton"
    },
    {
      "code": "021",
      "value_rep

In [172]:
import re

county_codes = []

for county in j['Data']:
    if re.search('Bronx|Kings|New York|Queens|Richmond', county['value_represented']):
        county_codes.append(county['code'])
        
print(county_codes)

['005', '047', '061', '081', '085']


## Transformed Datasets

In [59]:
air_quality.sample(10)

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
7817,386,Ozone (O3),Mean,ppb,CD,Borough Park (CD12),Summer 2012,2012-06-01,33.11
12579,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Williamsbridge and Baychester (CD12),Annual Average 2017,2017-01-01,18.35
3898,375,Nitrogen Dioxide (NO2),Mean,ppb,UHF34,Rockaways,Winter 2011-12,2011-12-01,15.89
8388,386,Ozone (O3),Mean,ppb,UHF42,Southeast Queens,Summer 2010,2010-06-01,34.29
154,641,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,South Beach - Tottenville,2013,2013-01-01,0.00
9370,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,Rockaway and Broad Channel (CD14),Annual Average 2015,2015-01-01,7.15
15484,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Bay Ridge and Dyker Heights (CD10),Summer 2020,2020-06-01,10.09
7888,386,Ozone (O3),Mean,ppb,CD,Flushing and Whitestone (CD7),Summer 2009,2009-06-01,26.29
15250,375,Nitrogen Dioxide (NO2),Mean,ppb,UHF42,Ridgewood - Forest Hills,Summer 2020,2020-06-01,10.49
9843,383,Sulfur Dioxide (SO2),Mean,ppb,UHF42,Southwest Queens,Winter 2014-15,2014-12-01,0.45


In [60]:
traffic_volume.head(15)

,RequestID,Boro,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,date_time
25543663,32417,Queens,20,67665,POINT (1004228.4823799994 215767.68782613552),31 STREET,34 Avenue,35 Avenue,SB,2020-11-22 23:45:00
20968423,32417,Queens,3,101621,POINT (1055667.922729934 216597.78334720692),DOUGLASTON PARKWAY,Maryland Road,Van Zandt Avenue,SB,2020-11-22 23:45:00
6368254,32417,Queens,27,148877,POINT (1015917.5752772316 218664.23589469263),23 AVENUE,Dead End,85 Street,WB,2020-11-22 23:45:00
7417214,32417,Queens,22,155846,POINT (1050166.9282559236 199291.28045636677),JAMAICA AVENUE,197 Street,198 Street,WB,2020-11-22 23:45:00
18926986,32417,Queens,5,76510,POINT (1017222.1345317845 216101.64900761063),31 AVENUE,87 Street,88 Street,WB,2020-11-22 23:45:00
13407547,32417,Queens,60,155773,POINT (1027092.2981505651 190399.9018293268),ATLANTIC AVENUE,97 Street,98 Street,WB,2020-11-22 23:45:00
7423333,32417,Queens,8,75313,POINT (1019421.0343045937 208800.348152701),51 AVENUE,90 Street,92 Street,EB,2020-11-22 23:45:00
18674125,32417,Queens,33,45497,POINT (1009925.0151697205 199046.51050050836),METROPOLITAN AVENUE,55 Street,56 Street,WB,2020-11-22 23:45:00
18956053,32417,Queens,6,145416,POINT (1046510.2750956557 205583.03286028266),UNION TURNPIKE,Dead End,Dead end,EB,2020-11-22 23:45:00
9171262,32417,Queens,5,101855,POINT (1057510.6819890984 218109.6099654521),MARATHON PARKWAY,Rushmore Avenue,Morenci Lane,SB,2020-11-22 23:45:00
